In [59]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import os

# upload datasets

In [2]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.MNIST(root='./data', train=True, transform=trans, download=True)
test_set = dset.MNIST(root='./data', train=False, transform=trans)
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=False)

# define neural networks

In [3]:
class AlexNet(nn.Module):   
    def __init__(self, num=10):
        super(AlexNet, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),   
            nn.MaxPool2d( kernel_size=2, stride=2),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(96, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=1),
        )
        self.classifier = nn.Sequential(
            nn.Linear(32*12*12,2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,num),
         
        )
    
    def forward(self, x):

        x = self.feature(x)
        x = x.view(-1,32*12*12)
        x = self.classifier(x)
        return x

# define train and test functions

In [4]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
#         loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target)
        train_losses.append(loss.data[0])
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('\rEpoch: {} {:.0f}%\t     Loss: {:.6f}'.format(
                epoch,
                100. * batch_idx / len(train_loader), loss.data[0]), end='')

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    acc=100. * float(correct.to(torch.device('cpu')).numpy())
    print('\nTest result: Average loss: {:.4f}, Accuracy: {:.4f}%\n'.format(
        test_loss, acc / len(test_loader.dataset)))
    
    test_accuracy.append(acc / len(test_loader.dataset))

# load model

In [5]:
model = AlexNet()


# check if gpu is available

In [ ]:
if torch.cuda.is_available():
    model.cuda()

# define learning rate of stochastic gradient descent

In [6]:
optimizer = optim.SGD(model.parameters(), lr=0.01)


# Perform TRAINING and TESTING iteratively

In [9]:
# train_losses = []
test_losses =[]
test_accuracy = []
# for epoch in range(1, 15):
#     train(epoch)
#     test()
test()

/var/folders/f6/8cblqgm51_l7vc0l47h_c2bw0000gn/T/ipykernel_34309/1755829714.py:26: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test result: Average loss: 2.3027, Accuracy: 10.3200%



# --------------------------------------------------------------------------------------


# FOR C++


# --------------------------------------------------------------------------------------
# .
# set path to folder where the weights should be downloaded

In [10]:
os.getcwd()

'/Users/daphneechabal/Library/CloudStorage/OneDrive-UvA/WORK/2.COMPARATIVE PAPER/falcon/scripts/MNIST_AlexNet'

In [11]:
path = "../../files/preload/MNIST/AlexNet/"
if not os.path.exists(path):
    print("THE OUTPUT FILE FOR THE WEIGHTS OF ALEXNET ON MNIST DOES NOT EXIST in /preload.")

# Take the very first test image and create the input file

In [102]:
flat_input = test_set[0][0].detach().numpy().flatten()

In [107]:
for i in range(3):
    np.savetxt(fname=f"{path}/input_0_{i}", delimiter=" ", X=flat_input)


# TAKE THE WEIGHTS AND BIAS OF THE REMAINING LAYERS WHICH ARE NOT RELU NOT MAXPOOL (only the layers that have weights) and create files

### Display layers first

In [67]:
model.parameters

<bound method Module.parameters of AlexNet(
  (feature): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(96, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=4608, out_features=2048, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=2048, out_features=1024, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=102

### create a file for each layer:
# remember: for i in range (3) is the generate identical files for each parties

In [108]:
for i in range(3):
    for name, param in model.named_parameters():
        n = name.split(".")
        if n[0] == "classifier":
            n[1] = int(n[1])+12
        new_name = n[2] + "_" + str(n[1]) + "_" + str(i)
        paramnp = param.detach().numpy().flatten()
        np.savetxt(fname=f"{path}/{new_name}", delimiter=" ", X=paramnp)
